In [1]:
import pandas as pd
from sqlalchemy import text, inspect
from aifinreport.database.connection import engine

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [3]:
# Get table schema
inspector = inspect(engine)
columns = inspector.get_columns('news_raw')

# Display as DataFrame
schema_df = pd.DataFrame([
    {
        'Column': col['name'],
        'Type': str(col['type']),
        'Nullable': col['nullable']
    }
    for col in columns
])

print("\n📋 news_raw Table Schema:")
schema_df


📋 news_raw Table Schema:


,Column,Type,Nullable
0,id,TEXT,False
1,published_utc,TIMESTAMP,False
2,title,TEXT,True
3,url,TEXT,True
4,description,TEXT,True
5,tickers,ARRAY,True
6,tags,ARRAY,True
7,published_date_utc,DATE,True
8,full_body,TEXT,True
9,full_body_chars,INTEGER,True


In [5]:
# Count total articles
query = text("SELECT COUNT(*) as total FROM news_raw")
with engine.connect() as conn:
    total = pd.read_sql(query, conn).iloc[0]['total']

print(f"\n📊 Total articles in database: {total:,}")

# Articles by ticker
query = text("""
    SELECT 
        unnest(tickers) as ticker,
        COUNT(*) as count
    FROM news_raw
    GROUP BY ticker
    ORDER BY count DESC
    LIMIT 10
""")

with engine.connect() as conn:
    ticker_counts = pd.read_sql(query, conn)

print("\n📈 Top 10 Tickers by Article Count:")
ticker_counts


📊 Total articles in database: 375

📈 Top 10 Tickers by Article Count:


,ticker,count
0,NVDA,242
1,TSLA,133


In [6]:
# Get 10 most recent articles
query = text("""
    SELECT *
    FROM news_raw
    ORDER BY published_utc DESC
    LIMIT 10
""")

with engine.connect() as conn:
    recent = pd.read_sql(query, conn)

print("\n🕐 10 Most Recent Articles:")
recent


🕐 10 Most Recent Articles:


,id,published_utc,title,url,description,tickers,tags,published_date_utc,full_body,full_body_chars,fetched_at,fetch_status,body_extractor,summary,validation_status,validation_reason,source,crawl_date
0,tiingo:89125185,2025-10-27 20:19:29+00:00,Nvidia GTC: What to expect as the AI roadshow comes to D.C.,https://finance.yahoo.com/news/nvidia-gtc-expect-ai-roadshow-201929344.html,"At Nvidia’s first Washington GTC, the company that made GPUs a religion is courting a new congre...",[NVDA],"[Digital Infrastructure, Jensen Huang, Keynote Address, Nvidia, Quantum Computing, Responsible A...",2025-10-27,"WASHINGTON — At Nvidia’s first-ever Washington GTC, the dress code will skew fewer hoodies, more...",2778,2025-10-29 06:22:07.997266+00:00,ok,trafilatura,"Huang’s keynote is expected to double as a civics lesson: a reminder that in an age of tariffs, ...",passed,ticker_in_title,finance.yahoo.com,2025-10-27 20:38:57.117323+00:00
1,tiingo:89106618,2025-10-27 12:00:00+00:00,Nvidia’s CEO Jensen Huang Summons AI Road Show to DC With US Nearing China Deal,https://finance.yahoo.com/news/nvidia-ceo-jensen-huang-summons-120000380.html,"Now, the same week President Donald Trump plans to meet China’s Xi Jinping to finalize a trade d...",[NVDA],"[Bloomberg, China, Donald Trump, Jensen Huang, Market Share, Nvidia, Stock, Technology, White Ho...",2025-10-27,(Bloomberg) -- Nvidia Corp. has spent the past three years ensnared in an escalating trade impas...,2776,2025-10-29 06:22:07.997266+00:00,ok,trafilatura,Huang will deliver his highly anticipated keynote for the first time in Washington at noon on Tu...,passed,ticker_in_title,finance.yahoo.com,2025-10-27 12:51:50.374122+00:00
2,tiingo:89099484,2025-10-27 06:08:14+00:00,NVIDIA's Quarterly Earnings Preview: What You Need to Know,https://finance.yahoo.com/news/nvidias-quarterly-earnings-preview-know-060814117.html,"Semiconductor giant NVIDIA is gearing up to release its Q3 results in the upcoming month, and an...",[NVDA],"[Nvidia, Semiconductor Company, Stock, Stock Prices, Technology]",2025-10-27,"Valued at a whopping $4.5 trillion by market cap, Santa Clara, California-based NVIDIA Corporati...",2775,2025-10-29 06:22:07.997266+00:00,ok,trafilatura,NVIDIA’s stock prices observed a marginal dip in the trading session following the release of it...,passed,ticker_in_title,finance.yahoo.com,2025-10-27 09:17:36.405762+00:00
3,tiingo:88472693,2025-10-08 19:35:02+00:00,Tesla just unveiled two cheaper EV models. Here’s how much they cost,https://finance.yahoo.com/news/tesla-just-unveiled-two-cheaper-193502183.html,New discount versions of the Model Y and Model 3 will be available in a few months.,[TSLA],"[Consumer Cyclical, Premium Models, President Donald Trump, Stock, Tesla]",2025-10-08,"Electric vehicle giant Tesla has unveiled new, cheaper models of its Model Y crossover SUV and M...",2486,2025-10-12 01:57:12.900222+00:00,ok,trafilatura,The Elon Musk-owned company just logged a record quarter of vehicle sales but it’s a benchmark t...,passed,ticker_in_title,finance.yahoo.com,2025-10-08 20:04:35.588427+00:00
4,tiingo:88471780,2025-10-08 19:05:49+00:00,Musk's xAI Funding Round Gets Boost From Nvidia,https://finance.yahoo.com/video/musks-xai-funding-round-gets-190549016.html,Elon Musk's artificial intelligence startup xAI is raising more financing than initially planned...,[NVDA],"[401K, Business, Economy, Finance, Financial Information, Investing, Investment Tools, Investor,...",2025-10-08,Elon Musk's artificial intelligence startup xAI is raising more financing than initially planned...,1634,2025-10-12 01:21:36.871740+00:00,ok,trafilatura,Elon Musk's artificial intelligence startup xAI is raising more financing than initially planned...,passed,ticker_in_title,finance.yahoo.com,2025-10-08 19:21:23.268996+00:00
5,tiingo:88471993,2025-10-08 18:58:36+00:00,"Tesla Unveils New Models for Under $40,000",https://finance.yahoo.com/video/tesla-unveils-models-under-40-185836118.html,Former Tesla board member Steve W